<a href="https://colab.research.google.com/github/terryzg/CDC-2024/blob/main/CDC_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import numpy as np
from google.colab import files
import plotly.express as px

In [41]:
df = pd.read_excel("Health_Science_Dataset.xlsx",skiprows = 0,header = 1)
df.head()

,Data As Of,Start Week,End Week,MMWRyear,MMWRweek,Week Ending Date,Group,Indicator,Jurisdiction,Age Group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Influenza Deaths,Pneumonia or Influenza,"Pneumonia, Influenza, or COVID-19 Deaths"
0,11/02/2023,12/29/2019,01/04/2020,2020,1,01/04/2020,By Week,Week-ending,United States,All Ages,0.0,60028.0,4102.0,432.0,4534.0,4534.0
1,11/02/2023,12/29/2019,01/04/2020,2020,1,01/04/2020,By Week,Week-ending,United States,0-17 years,0.0,667.0,19.0,22.0,41.0,41.0
2,11/02/2023,12/29/2019,01/04/2020,2020,1,01/04/2020,By Week,Week-ending,United States,18-64 years,0.0,14706.0,767.0,183.0,950.0,950.0
3,11/02/2023,12/29/2019,01/04/2020,2020,1,01/04/2020,By Week,Week-ending,United States,65 years and over,0.0,44655.0,3316.0,227.0,3543.0,3543.0
4,11/02/2023,12/29/2019,01/04/2020,2020,1,01/04/2020,By Week,Week-ending,Alabama,All Ages,0.0,1098.0,67.0,NaN,72.0,72.0


In [42]:
# Drop some columns
columns_to_drop = ['Data As Of', 'Indicator', 'Group', 'Pneumonia or Influenza', 'Pneumonia, Influenza, or COVID-19 Deaths']
df = df.drop(columns=columns_to_drop)

In [45]:
# Rename 'Jurisdiction' to 'State' and format for choropleth map
df_2 = df.rename(columns={'Jurisdiction': 'State'})

state_mapping = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'District of Columbia': 'DC',
    'United States': 'US',
    "HHS Region 1": "HHS1",
    "HHS Region 2": "HHS2",
    "HHS Region 3": "HHS3",
    "HHS Region 4": "HHS4",
    "HHS Region 5": "HHS5",
    "HHS Region 6": "HHS6",
    "HHS Region 7": "HHS7",
    "HHS Region 8": "HHS8",
    "HHS Region 9": "HHS9",
    "HHS Region 10": "HHS10",
}
df_2['State'] = df_2['State'].map(state_mapping)

In [46]:
# Convert 'Start Week' to datetime and keep as timestamp
df_2['Timestamp'] = pd.to_datetime(df['End Week'])
df_2 = df_2.drop(columns=['Start Week', 'End Week', 'Week Ending Date'])

In [48]:
# Function to fill missing values with mean of the age group for the state
def fill_missing(group):
    return group.fillna(group.mean())

In [51]:
# Fill empty entries
df_2 = df_2.replace('', np.nan)

numeric_columns = ['COVID-19 Deaths', 'Total Deaths', 'Pneumonia Deaths', 'Influenza Deaths']

# Apply the function to each group (State and Age Group)
df_2[numeric_columns] = df_2.groupby(['State', 'Age Group'])[numeric_columns].transform(fill_missing)

# Recalculate 'Total Deaths' if it's still NaN
df_2['Total Deaths'] = df_2[['COVID-19 Deaths', 'Pneumonia Deaths', 'Influenza Deaths']].sum(axis=1)
df_2['Pneumonia or Influenza'] = df_2[['Pneumonia Deaths', 'Influenza Deaths']].sum(axis=1)
df_2['Pneumonia or Influenza or Covid'] = df_2[['Pneumonia Deaths', 'Influenza Deaths','COVID-19 Deaths']].sum(axis=1)
df_2.dropna(inplace=True)

# Convert relevant columns to integer type
df_2[[*numeric_columns, 'Pneumonia or Influenza']] = df_2[[*numeric_columns, 'Pneumonia or Influenza']].astype(int)

In [61]:
df_2 = df_2.rename(columns={'MMWRyear': 'Year'}).rename(columns={'MMWRweek': 'Week'})

In [62]:
new_column_order = ["Year", "Timestamp", "Week","State", "Age Group",	"COVID-19 Deaths", "Pneumonia Deaths", "Influenza Deaths", "Total Deaths"]  # Desired column order
df_2 = df_2[new_column_order]
df_2

,Year,Timestamp,Week,State,Age Group,COVID-19 Deaths,Pneumonia Deaths,Influenza Deaths,Total Deaths
0,2020,2020-01-04,1,AK,0-17 years,0,0,0,0
1,2020,2020-01-04,1,AK,18-64 years,0,3,0,3
2,2020,2020-01-04,1,AK,65 years and over,0,13,0,13
3,2020,2020-01-04,1,AK,All Ages,0,18,0,18
4,2020,2020-01-04,1,AL,0-17 years,0,0,0,0
...,...,...,...,...,...,...,...,...,...
49595,2023,2023-10-28,43,WV,All Ages,10,21,0,31
49596,2023,2023-10-28,43,WY,0-17 years,0,0,0,0
49597,2023,2023-10-28,43,WY,18-64 years,0,0,0,0
49598,2023,2023-10-28,43,WY,65 years and over,15,16,0,31


In [54]:
df_description = df_2 [["COVID-19 Deaths", "Pneumonia Deaths", "Influenza Deaths","Total Deaths"]].describe().astype(int)
df_description = df_description.drop('count')
df_description

,COVID-19 Deaths,Pneumonia Deaths,Influenza Deaths,Total Deaths
mean,143,142,3,288
std,737,589,20,1317
min,0,0,0,0
25%,0,0,0,0
50%,22,24,0,51
75%,69,89,0,161
max,25974,16884,1048,42806


In [55]:
# Sort the dataframe
df_2 = df_2.sort_values(['Timestamp','State','Age Group'])

# Reset index
df_2 = df_2.reset_index(drop=True)

df_2.head()

,MMWRyear,Timestamp,MMWRweek,State,Age Group,COVID-19 Deaths,Pneumonia Deaths,Influenza Deaths,Total Deaths
0,2020,2020-01-04,1,AK,0-17 years,0,0,0,0
1,2020,2020-01-04,1,AK,18-64 years,0,3,0,3
2,2020,2020-01-04,1,AK,65 years and over,0,13,0,13
3,2020,2020-01-04,1,AK,All Ages,0,18,0,18
4,2020,2020-01-04,1,AL,0-17 years,0,0,0,0


In [69]:
df_without_US_REGION = df_2[~df_2['State'].isin(['US', 'HHS1', 'HHS2', 'HHS3', 'HHS4', 'HHS5', 'HHS6', 'HHS7', 'HHS8', 'HHS9', 'HHS10'])]

In [ ]:
# Group data by timestamp and death type for a line chart
df_time_deaths = df_without_US_REGION.groupby(['Timestamp'])[['COVID-19 Deaths', 'Pneumonia Deaths', 'Influenza Deaths']].sum().reset_index()

# Create a line chart using plotly
fig = px.line(df_time_deaths, x='Timestamp', y=['COVID-19 Deaths', 'Pneumonia Deaths', 'Influenza Deaths'],
             title='Deaths Over Time')
fig.show()


In [57]:
df_filtered_age = df_2[df_2['Age Group'] != 'All Ages']

# Group by Age Group and sum the deaths
age_group_deaths = df_filtered_age.groupby('Age Group')[['COVID-19 Deaths', 'Pneumonia Deaths', 'Influenza Deaths']].sum().reset_index()

# Create a bar chart with Plotly Express
fig = px.bar(age_group_deaths,
             x='Age Group',
             y=['COVID-19 Deaths', 'Pneumonia Deaths', 'Influenza Deaths'],
             barmode='group',
             title='Deaths by Age Group and Cause of Death')

# Show the figure
fig.show()

In [ ]:
# Columns for deaths and week
# Drop unnecessary columns
df_dropped = df_without_US_REGION.drop(columns=['MMWRyear', 'Timestamp', 'State', 'Age Group', 'Pneumonia or Influenza', 'Total Deaths'])

# Specify the death columns
death_columns = ['COVID-19 Deaths', 'Pneumonia Deaths', 'Influenza Deaths']

# Group by MMWRweek and sum the specified columns
grouped_df = df_dropped.groupby('MMWRweek')[death_columns].sum().reset_index()
grouped_df

,MMWRweek,COVID-19 Deaths,Pneumonia Deaths,Influenza Deaths
0,1,88402,75347,2191
1,2,95093,78324,1696
2,3,96176,77326,1285
3,4,87461,72140,1187
4,5,75935,66700,966
5,6,63268,58405,996
6,7,49878,51555,1035
7,8,40542,45094,1173
8,9,31869,40760,1325
9,10,28705,36820,1246


In [ ]:
df_deaths_grouped_by_state = df_without_US_REGION.groupby(['State'])[['COVID-19 Deaths', 'Pneumonia Deaths', 'Influenza Deaths', 'Pneumonia or Influenza', 'Pneumonia or Influenza or Covid', 'Total Deaths']].sum().astype(int)
df_deaths_grouped_by_state


,COVID-19 Deaths,Pneumonia Deaths,Influenza Deaths,Pneumonia or Influenza,Pneumonia or Influenza or Covid,Total Deaths
State,,,,,,
AK,5137,6845,0,6845,12378,12106
AL,50651,38021,255,38276,89009,88937
AR,30625,28292,156,28448,59234,59090
AZ,65097,60573,275,60848,126019,125947
CA,226012,253490,6112,259602,485757,485614
CO,33591,30941,299,31240,64929,64835
CT,31527,22609,102,22711,54327,54244
DC,6303,8717,0,8717,15442,15110
DE,10731,8788,10,8798,19890,19705


In [71]:
# Deaths grouped by year and death type

df_grouped_year = df_without_US_REGION.groupby(['Year'])[['COVID-19 Deaths', 'Pneumonia Deaths', 'Influenza Deaths']].sum().astype(int).reset_index()
df_grouped_year


,Year,COVID-19 Deaths,Pneumonia Deaths,Influenza Deaths
0,2020,779244,697571,12668
1,2021,910998,807321,4357
2,2022,521065,532969,14774
3,2023,221357,308917,7702


In [73]:
# Calculate the sum of each column except 'Year'
column_sums = df_grouped_year.loc[:, df_grouped_year.columns != 'Year'].sum()

print(column_sums)


COVID-19 Deaths     2432664
Pneumonia Deaths    2346778
Influenza Deaths      39501
dtype: int64
